<a href="https://colab.research.google.com/github/akramIOT/DPO_Versus_RLHF_MISTRAL-7B/blob/main/RLHF_DPO_mistralai_Mistral_7B_v0_1_execution_Akram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement Learning from Human Feedback

RLHF basically (and I say basically rather cheekily) this process:

### 1) 🏁 Start with a Model

- This could be a base model, or a model that's gone through some supervised fine-tuning. For example: instruction-tuning, chat-tuning, summarization-tuning, etc.

- Give the model prompts and have it generate multiple completions.

### 2) 🗣️ Collect Human Feedback

Ask humans to evaluate these responses, selecting the preferred ones, or ranking them according to certain criteria.

### 3) 💳 Train a Reward Model

- Use the collected human feedback to train a reward model.

- This model learns to predict the human preference between pairs of responses.

- The reward model approximates human judgments, assigning higher scores to more preferable responses.

🚨 **Important**: The reward model must output a scalar value score (ie, just a number) to each output generated by the LLM. This score is used for reinforcement learning, because it gives measures the output's alignment with human preferences.

### 4) 🤖 Do Reinforcement Learning

<img src="https://user-images.githubusercontent.com/17574157/213037462-5dd556de-3afa-4842-b546-0fc90e799249.png">


- Make a duplicate copy of the language model and freeze it's weights. Now you have two models: one whose weights will be updated via back prop, and one that will not be updated. The frozen LM acts as a quality control, so the trainable LM doesn't try to cheat the system.

- Generate completions from both the models. Measure how much the trainable LM's text predictions drift from the frozen one using KL divergence loss.

- Add that KL divergence loss to the reward from the reward model to keep things on track. Use the trained reward model to fine-tune the base model further.

- This step involves reinforcement learning algorithms, usually Proximal Policy Optimization (PPO).

- Optimize the policy of the base model to generate responses that maximize the predicted reward from the reward model.


## PPO Overview

This involves three steps which, to quote Hugging Face, is as follows: Rollout, Evaluate, and Optimize.

<img src="https://camo.githubusercontent.com/14c39f51b64e2fc3f1ed9da4af3f9027034544551aed3045d7a5930d603e233a/68747470733a2f2f68756767696e67666163652e636f2f64617461736574732f74726c2d696e7465726e616c2d74657374696e672f6578616d706c652d696d616765732f7265736f6c76652f6d61696e2f696d616765732f74726c5f6f766572766965772e706e67">

### 🧻 **Rollout**

The language model generates completions for some prompt or query.  Basically, generate tokens that can complete the sequence.

You then compute the reward for the completions and evaluate.

### 🔬 **Evaluate**

The query and response can be evaluated using a function, model, human feedback, or a combination.

This process should produce a single numerical value for each query/response pair.

You can check the scores of those tokens using the Reward Model.

### 💪🏽 **Optimize**

Using a trained model and a reference model (usually the pre-trained model before fine-tuning) calculate the log probabilities of the tokens in the sequences.

The KL divergence between the two outputs is then used as an additional reward signal to ensure that the generated responses do not deviate too far from the reference language model.

The active language model is trained with PPO and updated based on the scores and the generations of the reference model, which is the original model before RLHF.

# 🎯 DPO Overview

References:  https://www.summarize.tech/www.youtube.com/watch?v=pzh2oc6shic

STANFORD UNIVERSOTY  SAIL: https://aibusiness.com/nlp/stanford-s-new-ai-training-method-fine-tunes-better-than-rlhf

The core insight is to derive a loss function that directly solves the policy optimization and reward modeling steps simultaneously.

<img src="https://pbs.twimg.com/media/GBGxPDTa0AEyjUd?format=jpg">


##  Detailed Explanation the loss function (KL LOSS and other Loss functions)

References:  https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence

https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/

https://arxiv.org/pdf/2305.18290.pdf

https://arxiv.org/abs/2402.13228


<img src="https://private-user-images.githubusercontent.com/1131538/301539636-1c4683af-7c65-4207-bd20-e2fc580a9542.png?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3MDY5MDIxMzksIm5iZiI6MTcwNjkwMTgzOSwicGF0aCI6Ii8xMTMxNTM4LzMwMTUzOTYzNi0xYzQ2ODNhZi03YzY1LTQyMDctYmQyMC1lMmZjNTgwYTk1NDIucG5nP1gtQW16LUFsZ29yaXRobT1BV1M0LUhNQUMtU0hBMjU2JlgtQW16LUNyZWRlbnRpYWw9QUtJQVZDT0RZTFNBNTNQUUs0WkElMkYyMDI0MDIwMiUyRnVzLWVhc3QtMSUyRnMzJTJGYXdzNF9yZXF1ZXN0JlgtQW16LURhdGU9MjAyNDAyMDJUMTkyMzU5WiZYLUFtei1FeHBpcmVzPTMwMCZYLUFtei1TaWduYXR1cmU9NTlkMmI5OTQzN2U2NzA4ZjQ1ZjQ5MzQ2NjQzZjJkYTQ5YjYwNjI1OTM0NjZmMDZlMDdkZTAxNmM4YThmYzcxZSZYLUFtei1TaWduZWRIZWFkZXJzPWhvc3QmYWN0b3JfaWQ9MCZrZXlfaWQ9MCZyZXBvX2lkPTAifQ._b_t5lTbTsCDI_HjJUdfJUq04WgL9vbY3UNNsgVJYm0">


1) Loss function of policy model with respect to reference model.

2) Expectation of the dataset with `x` samples of $y_{w}$ (chosen) and $y_{l}$ rejected outputs.

3) The logarithmic of the sigmoid applied to the argument.
  - In torch: `F.logsigmoid`
  - This will scale the result between 0 and 1, providing a probabilistic interpretation.

4) $\beta$: Hyperparameter that weights the importance of the deviation between the policy model and the reference model.

5) Log Probability ratio of the policy model’s probability of choosing the same  $y_{w}$  given the input `x`, divided by the reference model’s probability of choosing the same $y_{w}$  given the same input `x`.
 - These ratios indicate how much more or less likely the policy model is to choose a particular action compared to the reference model.


### DPO Pipieline

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*AqKOT0pxzi5kOgiobb-Fvg.png">

- Start, again, with two copies of your model. One with frozen weights and one with trainable weights

- Every sample gets a score from both the trainee LM and its frozen twin, based on how likely they are to spit out the expected response.

- Scores for chosen versus rejected responses to a given prompt are calculated using a simple binary classification approach.

- The model is trained to learn "good" and "bad" responses based on human preference data.

-  The chosen/rejected score is the product of probabilities assigned to each token in the chosen response sequence. This score reflects the model's assessment of the response's quality, with higher scores indicating a response more aligned with human preferences.

- The multiplication of probabilities across the sequence yields the final score for the response, which is used to guide the optimization of the model.

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*-l_uEgU-Qq-32kw4DgDgxg.png">


- The loss on the sample in DPO is calculated using the standard cross-entropy loss function. The model produces a probability distribution over the possible responses, and the cross-entropy loss measures the difference between the predicted probability distribution and the actual distribution (the ground truth).

- In the context of DPO, the actual distribution is based on human preferences, with the chosen response being the positive class and the rejected response being the negative class.

- The loss is minimized when the model's predictions align closely with the human-provided labels, effectively training the model to predict human preferences with higher accuracy.

##References:

Direct Preference Optimization can achieve performance levels that are equivalent to, and sometimes surpass, those attainable with RLHF and PPO, as evidenced by the results of the experiments reported in Direct Preference Optimization: Your Language Model is Secretly a Reward Model.

Stanford Univ and CZ BioHub, SAIL Research paper on DPO:
 https://arxiv.org/pdf/2305.18290.pdf

# DPO vs RLHF


<img src="https://i.ibb.co/mvFwnWc/dpo.gif">

# Akram Sheriff: Key Learnings Summary on this DPO execution project with LLM

1) DPO takes a simpler execution path to aligning LLMs than RLHF technique.

2) RLHF or  RLAIF  requires multiple stages of  custom data pipeline to be built in the engineering execution phase with proper selection of the Config parameters and is Not a  Trivial Task, $$$ cost and  time is involved.

3) Typically RLHF pipeline involves multiple stages, including training a reward model(RM) on human feedback, using this model to generate rewards for policy training, and then using reinforcement learning algorithms to adjust the language model.

4) LLM itself acts as the Secret Reward Model and hence  DPO is  good  enough technique

5) RLHF or PPO is computationally intensive and requires careful hyperparameter tuning.

6) The DPO pipeline directly applies a classification loss to the language model, using human preference data to guide the optimization.

7) This nullifies the intricate need for optimizing of RL algorithms and the meticulous adjustment of hyperparameters. As a result, DPO runs smoother and demands less computational power than RLHF, which often requires model sampling during its fine-tuning phase and can stumble into instability and higher computational needs.

8) In summary, DPO offers a streamlined and efficient alternative to the more *cumbersome* RLHF approach or  RLAIF technique.


In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install -q accelerate
!pip install -q https://pypi.org/simple/ bitsandbytes

In [ ]:
import torch
import os
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from trl import DPOTrainer


## Local MACOS M2 based Jupyter Runtime used to run this  DPO workfload and hence a Colab Widget is used to view the resources data via widget manager.

#from google.colab import output
#output.enable_custom_widget_manager()

The `distilabel-intel-orca-dpo-pairs dataset` is a  refinement of the [Intel/orca_dpo_pairs](https://huggingface.co/datasets/Intel/orca_dpo_pairs) through [distilabel](https://github.com/argilla-io/distilabel), aligning more closely with AI community needs and model refinement practices.

Learn more about the dataset [here](https://huggingface.co/datasets/argilla/distilabel-intel-orca-dpo-pairs)

In [ ]:
dataset = load_dataset("argilla/distilabel-intel-orca-dpo-pairs", split="train")

In [ ]:
dataset = dataset.filter(
    lambda r:
        r["status"] != "tie" and
        r["chosen_score"] >= 8 and
        not r["in_gsm8k_train"]
)

# You need three features in your dataset  Mandatory  for  DPO mechanism

 - `prompt` -  the context prompt which is given to a model at inference time for text generation

 - `chosen` -  the preferred generated response to the prompt

 - `rejected` - the dispreferred response to the prompt

In [ ]:
def insert_prompts(example):
    """
    Inserts data into system_prompt and user_prompt placeholders.

    Args:
        example (dict): A dictionary representing a record in the dataset.

    Returns:
        dict: The modified record with the completed prompt.
    """
    system_prompt_template = "### System:\n{system_prompt}\n### User:\n{user_prompt}\n### Assistant:"
    system_prompt = example["system"]
    user_prompt = example["input"]


    filled_prompt = system_prompt_template.format(system_prompt=system_prompt, user_prompt=user_prompt)

    example["prompt"] = filled_prompt
    return example

dataset = dataset.map(insert_prompts)

In [ ]:
dataset = dataset.shuffle(seed=42).train_test_split(seed=42, test_size=.3)

In [ ]:
dataset = dataset.remove_columns(['generations', 'order', 'labelling_model', 'labelling_prompt', 'raw_labelling_response', 'rating', 'rationale', 'status', 'original_chosen', 'original_rejected', 'chosen_score', 'in_gsm8k_train'])

In [ ]:
train_data = dataset["train"]

test_data = dataset["test"]

In [ ]:
'''model_name = "Deci/DeciLM-7B-instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
    trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token

tokenizer.pad_token_id =  tokenizer.eos_token_id
'''

In [ ]:
## https://huggingface.co/mistralai/Mistral-7B-v0.1   -  Reference PreTrained Model

model_name = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
    trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token

tokenizer.pad_token_id =  tokenizer.eos_token_id

# Instantiate policy model

In [ ]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
)

policy_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

policy_model = prepare_model_for_kbit_training(policy_model)

policy_model.config.pad_token_id = tokenizer.pad_token_id

policy_model.config.use_cache = False

In [ ]:
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["gate_proj", "down_proj", "up_proj"]
)

# Instantiate reference model

In [ ]:
reference_model_name = "mistralai/Mistral-7B-v0.1"

bnb_config_ref = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
)

reference_model = AutoModelForCausalLM.from_pretrained(
    reference_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code = True,
    low_cpu_mem_usage=True,
)

# Set training arguments

In [ ]:
training_arguments = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_32bit",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=2,
        log_level="debug",
        save_steps=10,
        logging_steps=10,
        learning_rate=5e-7,
        eval_steps=20,
        #num_train_epochs=1,
        max_steps=100,
        warmup_steps=20,
        lr_scheduler_type="cosine",
        remove_unused_columns=True
)

# Train!

In [ ]:
trainer = DPOTrainer(
    policy_model,
   # reference_model,
    max_length=8196,
    max_prompt_length=4096,
    args=training_arguments,
    beta=0.1,
    peft_config=peft_config,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
)

trainer.train()